In [88]:
import pandas as pd 
import matplotlib.pyplot as plt
from scipy.stats import entropy
import numpy as np
import altair as alt

In [85]:
df_yellowstone = pd.read_csv("species_yellowstone.csv",skiprows=1)
df_acad = pd.read_csv("ACAD_species.csv",skiprows=1)
df_agfo = pd.read_csv("AGFO_species.csv")
df_yellowstone = df_yellowstone.drop(['Region', 'Internal Park Tags', 'ExternalLinks', 'Ozone', 'Park Tags', 'Nativeness Tags'], axis=1)
df_acad = df_acad.drop(['Region', 'Internal Park Tags', 'ExternalLinks', 'Ozone', 'Park Tags', 'Nativeness Tags'], axis=1)
df_agfo = df_agfo.drop(['Region', 'Internal Park Tags', 'ExternalLinks', 'Ozone', 'Park Tags', 'Nativeness Tags'], axis = 1)

In [91]:
species_richness_yellowstone = df_yellowstone['Scientific Name'].nunique()
species_richess_acad = df_acad['Scientific Name'].nunique()
species_richess_agfo = df_agfo['Scientific Name'].nunique()

species_data = pd.DataFrame({
    'Park': ['Yellowstone', 'Acadia', 'Agate Fossil Beds'],
    'Species Richness': [yellowstone_species_richness, acad_species_richness, agfo_species_richness]
})

chart = alt.Chart(species_data).mark_bar().encode(
    x=alt.X('Park', title='National Parks'),
    y=alt.Y('Species Richness', title='Number of Unique Species'),
    color=alt.Color('Park', legend=None)  # Optional color coding by park
).properties(
    title='Species Richness (Uniqueness) Comparison Across Parks'
)

# Display the chart
chart.show()

alt.Chart(...)